In [1]:
%%configure -f
{
    "conf":{
        "spark.executor.instances": "4",
        "spark.executor.memory": "2g",
        "spark.executor.cores": "1"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
729,application_1765289937462_0722,pyspark,idle,Link,Link,None,
735,application_1765289937462_0728,pyspark,idle,Link,Link,None,
737,application_1765289937462_0730,pyspark,idle,Link,Link,None,
739,application_1765289937462_0732,pyspark,idle,Link,Link,None,
740,application_1765289937462_0733,pyspark,idle,Link,Link,None,
747,application_1765289937462_0740,pyspark,idle,Link,Link,None,
748,application_1765289937462_0741,pyspark,idle,Link,Link,None,
749,application_1765289937462_0742,pyspark,idle,Link,Link,None,
750,application_1765289937462_0743,pyspark,idle,Link,Link,None,


In [2]:
from pyspark.sql import SparkSession
import csv
from io import StringIO

def parse_csv(line):
    return next(csv.reader([line]))
sc = SparkSession \
    .builder \
    .appName("RDD query 3 execution") \
    .getOrCreate() \
    .sparkContext

rdd1 = sc.textFile("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2010_2019.csv")\
            .map(parse_csv)

rdd2 = sc.textFile("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2020_2025.csv")\
            .map(parse_csv)

crime_rdd_raw = rdd1.union(rdd2)

# s3 path of MO codes file
mo_codes_path = "s3://initial-notebook-data-bucket-dblab-905418150721/project_data/MO_codes.txt"

mo_dict = sc.textFile(mo_codes_path) \
            .map(lambda line: line.strip()) \
            .filter(lambda line: line != "" and not line.startswith("#")) \
            .map(lambda line: line.split(None, 1)) \
            .map(lambda parts: (parts[0].strip(), parts[1].strip() if len(parts) > 1 else "")) \
            .collectAsMap()
# mo_dict: { "1234": "DESCRIPTION", ... }

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
751,application_1765289937462_0744,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
import time

MOCODES_IDX = 10


# drop header rows and any malformed rows
crime_rdd = crime_rdd_raw.filter(
    lambda row: len(row) > MOCODES_IDX and row[0] != "DR_NO"
)

# explode mocodes: one record per single code
mocodes_rdd = crime_rdd.flatMap(
    lambda row: [
        code for code in (row[MOCODES_IDX] or "").split() 
        if code
    ]
)

mo_counts_rdd = mocodes_rdd.map(lambda code: (code, 1)) \
                           .reduceByKey(lambda a, b: a + b)
# (MO_CODE, count)

start = time.time()

bc_mo_dict = sc.broadcast(mo_dict)

# (code, count, description)
result_rdd = mo_counts_rdd.map(
    lambda kv: (kv[0], kv[1], bc_mo_dict.value.get(kv[0], "UNKNOWN"))
)

# sort by count descending
result_sorted = result_rdd.sortBy(lambda x: -x[1])

# e.g. show top 10
for code, cnt, desc in result_sorted.take(10):
    print(code, cnt, desc)

end = time.time()
print("Execution time:", end - start, "seconds")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0344 1002900 Removes vict property
1822 548422 Stranger
0416 404773 Hit-Hit w/ weapon
0329 377536 Vandalized
0913 278618 Victim knew Suspect
2000 256188 Domestic violence
1300 219082 Vehicle involved
0400 213165 Force used
1402 177470 Evidence Booked (any crime)
1609 131229 Smashed
Execution time: 9.286213874816895 seconds